<p style="font-size:small; color:gray;"> Author: 鄭永誠, Year: 2024 </p>

# 一些文檔處理工具基於 llama_index
----------

## 基礎改念 (複習):
- **token / tokenization** ➡️ token是LLM在處理文字時的最小單位，可以理解就像是一個個單字，留意不同LLM會用[不同詞彙表](https://huggingface.co/docs/transformers/en/tokenizer_summary)切token

- **embedding** ➡️ 把這些token轉成一組高維向量，而這個向量用來表示這個句子的涵義，要留意不同模型能吃的token大小有異

- **chunk** ➡️ 分塊，當資料量太大時，我們會將其切成一個一個分塊(chunks)

- **parse / parsing** ➡️ 把句子解析、轉換成更好理解的格式，像是各類文檔語法結構調整、抽取訊息等，像是從html, json...轉成更好使用的格式

- **extractor** ➡️ 文本在存儲之前，我們可以透過LLM先一步去從中識別和提取特定的信息，如關鍵字識別、主題建模、摘要、建立相關問題...

- **pipeline** ➡️ 上面講了很多處理的流程，我們可以將其串在一起，建立所謂的資料處理流程(pipeline)，此處會講基於llamaindex的實踐方法，(下面統稱transormers)


## 最終完整流程(Data Ingestion)可能包含

- **Loaders** ➡️ 允許與外部源集成以上傳信息

- **transormers** ➡️ 資料處理流程，如 parse, split to chunk, extract, embedding... 等多種流程

- **Vector Stores** ➡️ 將資訊存入向量資料庫

- **Retrievers** ➡️ 用於信息檢索的組件，從大規模文本數據集中檢索相關信息

- **LLM Agent / Tools** ➡️ 各個處理問題的Agent、LLM模型或各種工具

- **Memories** ➡️  記錄對話

- **Output Parsers** ➡️ 把結果轉換成需求的格式，如json...


-------------
## ✏️ IngestionPipeline 


In [ ]:
""" """
%pip install llama_index.core -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
""" 這邊以 MarkdownNodeParser 為例，可以參考其他的NodeParser """
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core import Document
import markdown

parser = MarkdownNodeParser()

# 這邊我們先定義一個讀取markdown文件的函數
def read_markdown_file(filename):
    # Create a Path object for the markdown file
    file_path = Path(filename)
    
    # Open and read the file content
    with file_path.open('r', encoding='utf-8') as file:
        content = file.read()
    
    return content

# 讀取我們的README.md文件
content = read_markdown_file('README.md')
markdown_content = markdown.markdown(content)


# 注意，node_parser不是直接吃文本，而是吃list of Document物件
documents = Document(text=markdown_content)  # id_ 可選，可以設置為任何標識符


# 我直接拿我們的README.md來做範例
nodes = parser.get_nodes_from_documents([documents])

# 留意我因為文檔太短，所以只有一個node
print("\n分割後的文檔：")
for i, node in enumerate(nodes):
    print(f'[part_{i}]：{node.text}')


分割後的文檔：
[part_0]：<div align="center">
  <h1>🤖 大型語言模型入門教學 中文分享整理  💻</h1>
  <p align="center">
    ✍️ <a href="https://hackmd.io/@pputzh5cRhi6gZI0csfiyA/H1ejIyxHR"> 作者: 鄭永誠</a> • 
    ✉️ <a href="mailto:jason0304050607@gmail.com">信箱</a> • 
    🧑‍🤝‍🧑 <a href="https://www.dalabx.com.tw//"> 合作夥伴: 紫式大數據決策 </a> • 
    👫 <a href="https://moraleai.com/"> 我的朋朋: Morale AI </a> 
  </p>
</div>
<p><br/></p>
<p>內容簡介:
1. 🍻 <strong>LLM基礎改念:</strong> 我會整理一些LLM的需求知識，但git上是實作資源為主不會講述太多，有興趣請密我
2. 🛠️ <strong>LLM相關工具:</strong> 以下內容全基於python實踐，同時會分享相關資源、套件、開源API...
3. 💬 <strong>LLM系統架構:</strong> 會帶你由淺入深，慢慢了解部屬LLM系統(多Agent)的方向和一些好用工具</p>
<p>這個分享內容宗旨:
1. 🧩 <strong>讓你好上手:</strong> 提供最簡單的、盡可能可複製即用的code，讓新手也能盡可能快速入門 (而且是中文XD)
2. 🎈 <strong>讓你免費玩:</strong> 全基於開源資源，讓你能夠無痛體驗LLM的功能和操作
3. 😊 <strong>讓你喜歡上:</strong> 盡量提供簡單有趣的小例子，讓你也能喜歡LLM可帶來的運用</p>
<p>範例使用版本/輔助工具:
- Python 3.12.4
- 語言模型主要使用 llama-3.1-70b-versatile
- 個人主要使用 IDE: VScode
- 搭配工具 寫程式大幫手 <a href="https://github.com/features/copilot">Copilot</a>
- 其他: 使用 <a hre

In [7]:
""" 安裝llamaindex """
# %pip uninstall llama_index -q
%pip install llama-index -q
%pip show llama_index

Note: you may need to restart the kernel to use updated packages.
Name: llama-index
Version: 0.10.64
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: c:\Users\PipiHi\Desktop\KM\llm-course-zh\.venv\Lib\site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install llama-index-core -q
%pip install llama-index-llms-openai -q
%pip install llama-index-llms-replicate -q
%pip install llama-index-embeddings-huggingface -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
documents_example = [
    {
        'title': 'Alice', 
        'content': 'Alice is very adaptable and can handle unexpected challenges with ease.', 
        'job': ['Engineer', 'Designer']
    }, 
    {
        'title': 'Bob',
        'content': 'Bob is a natural leader with strong communication skills.',
        'job': ['Teacher', 'Writer']
    }, 
    {
        'title': 'Charlie',
        'content': 'Charlie is a charismatic individual who easily connects with others', 
        'job': ['Doctor', 'Researcher']
    }
]


------------
## ✏️ SentenceSplitter - chunk

In [20]:
%pip show llama_index

Name: llama-index
Version: 0.10.64
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: c:\Users\PipiHi\Desktop\KM\llm-course-zh\.venv\Lib\site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [16]:
""" 網路上llama_index舊版的是用 llama_index.XXXX ，新版用llama_index.core.XXXX """
import textwrap
from llama_index.core import Document
from llama_index.core.text_splitter import SentenceSplitter

# 自定義一個文件
documents = "義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪跟維爾康在南極匯合。"

print("原本文檔：")
print(textwrap.fill(documents, width=50))

node_parser  = SentenceSplitter(
    chunk_size=100, # 設定每個chunk的大小
    chunk_overlap=5, # 設定chunk之間的重疊大小
    tokenizer= None, # 設定分詞器
    paragraph_separator="\n\n", # 設定段落之間的分隔符號
    separator=" ", # 用於拆分句子的預設的分隔字元
    secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?' # 用於分割句子的備份正規表示式
)

nodes = node_parser.get_nodes_from_documents(
    [Document(text=documents)], show_progress=False
)
print("\n分割後的文檔：")
for i, node in enumerate(nodes):
    print(f'[part_{i}]：{node.text}')


原本文檔：
義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到
挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至
於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲
野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪
跟維爾康在南極匯合。

分割後的文檔：
[part_0]：義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。
[part_1]：你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至於對整個太平洋，和充電器的核污染。再或者說透
[part_2]：者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈
[part_3]：是否有沈澱物，都不會影響到沃爾瑪跟維爾康在南極匯合。


-------------
## ✏️ SentenceSplitter - parser
有許多用來處理各種文檔的parser，詳見官方   
https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/
""" """
%pip install llama_index.core -q
""" 這邊以 MarkdownNodeParser 為例，可以參考其他的NodeParser """
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core import Document
import markdown

parser = MarkdownNodeParser()

# 這邊我們先定義一個讀取markdown文件的函數
def read_markdown_file(filename):
    # Create a Path object for the markdown file
    file_path = Path(filename)
    
    # Open and read the file content
    with file_path.open('r', encoding='utf-8') as file:
        content = file.read()
    
    return content

# 讀取我們的README.md文件
content = read_markdown_file('README.md')
markdown_content = markdown.markdown(content)


# 注意，node_parser不是直接吃文本，而是吃list of Document物件
documents = Document(text=markdown_content)  # id_ 可選，可以設置為任何標識符


# 我直接拿我們的README.md來做範例
nodes = parser.get_nodes_from_documents([documents])

# 留意我因為文檔太短，所以只有一個node
print("\n分割後的文檔：")
for i, node in enumerate(nodes):
    print(f'[part_{i}]：{node.text}')

-------------
## ✏️ SentenceSplitter - parser
有許多用來處理各種文檔的parser，詳見官方   
https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

In [ ]:
""" """
%pip install llama_index.core -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
""" 這邊以 MarkdownNodeParser 為例，可以參考其他的NodeParser """
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core import Document
import markdown

parser = MarkdownNodeParser()

# 這邊我們先定義一個讀取markdown文件的函數
def read_markdown_file(filename):
    # Create a Path object for the markdown file
    file_path = Path(filename)
    
    # Open and read the file content
    with file_path.open('r', encoding='utf-8') as file:
        content = file.read()
    
    return content

# 讀取我們的README.md文件
content = read_markdown_file('README.md')
markdown_content = markdown.markdown(content)


# 注意，node_parser不是直接吃文本，而是吃list of Document物件
documents = Document(text=markdown_content)  # id_ 可選，可以設置為任何標識符


# 我直接拿我們的README.md來做範例
nodes = parser.get_nodes_from_documents([documents])

# 留意我因為文檔太短，所以只有一個node
print("\n分割後的文檔：")
for i, node in enumerate(nodes):
    print(f'[part_{i}]：{node.text}')

NameError: name 'Path' is not defined

------------
## ✏️ sentence_transformers - Embedding

In [15]:
""" Embedding方法示意，可參考C4內容 """
from sentence_transformers import SentenceTransformer


# 載入模型 (已選擇中文擅長模型)
model = SentenceTransformer('DMetaSoul/sbert-chinese-general-v2')  

text = node_parser.get_nodes_from_documents([Document(text=documents)], show_progress=False)[0].text
print("拿一個切完的chunk範例: ", text)

embedding = model.encode(text, convert_to_tensor=False)
print("\n句子Embedding 後結果: ", embedding)

拿一個切完的chunk範例:  義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。

句子Embedding 後結果:  [ 2.28983879e-01  2.56072372e-01 -9.01247978e-01  2.16540433e-02
  1.61327943e-01 -1.41556514e-02  3.84209663e-01 -1.99013099e-01
 -1.55311847e+00  6.03017390e-01  2.07268342e-01 -3.16158026e-01
 -5.27468443e-01 -7.72385418e-01 -7.33864367e-01 -7.74600148e-01
  1.51136622e-01 -3.20973635e-01 -3.76673907e-01 -9.55326915e-01
 -7.29235470e-01  2.18102023e-01 -4.98580933e-01 -1.31339148e-01
  7.16233611e-01  3.13105881e-01  3.83338720e-01 -9.17241454e-01
  5.43770613e-03  4.03267086e-01  6.08780324e-01 -1.19084668e+00
 -1.05522287e+00  7.90530562e-01  1.84666634e-01  4.99955505e-01
  3.46025735e-01  7.76784360e-01  1.76949695e-01 -9.94610190e-01
  9.47135165e-02 -1.31648213e-01 -4.64734465e-01  1.02906168e+00
  1.52269915e-01  5.49417913e-01  2.23850280e-01  5.11151791e-01
 -9.47740257e-01  4.94052619e-01  9.64427516e-02  6.79402590e+00
 -4.76693422e-01  1.04922771e+00 -7.90978849e-01  9.72206533e-

In [28]:
# 補充: 當然你也可以利用langchain_community或langchain_community 使用HuggineFace上的其他模型，例如：

# %pip install llama-index-embeddings-huggingface -q
# %pip install llama-index-embeddings-instructor -q

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 使用HuggingFaceEmbedding上的模型
model_from_hf = HuggingFaceEmbeddings(model_name="DMetaSoul/Dmeta-embedding-zh-small")

# 注意，HuggingFaceEmbeddings舊的獲取向量函數是get_text_embedding不是embed_query
embedding = model_from_hf.embed_query(text)
print("句子Embedding 後結果: \n", embedding)

句子Embedding 後結果: 
 [-0.011561818420886993, -0.027052123099565506, -0.046663232147693634, 0.011066941544413567, -0.004675247240811586, -0.018668046221137047, 0.0036121418233960867, -0.06413348019123077, 0.002792960498481989, -0.02290388196706772, 0.002651661168783903, 0.03054366260766983, -2.5104986889346037e-06, -0.00016913010040298104, -0.059822823852300644, -0.030154917389154434, -0.0007710297359153628, -0.08172184973955154, 0.008165168575942516, -0.011992831714451313, -0.020475028082728386, 0.0741274282336235, 0.003467828268185258, -0.0053542302921414375, -0.038078323006629944, 0.02178216725587845, 0.026178831234574318, 0.0022571897134184837, -0.06470903009176254, -0.007674640975892544, -0.00895301066339016, 0.026491597294807434, -0.014211810193955898, -0.010663865134119987, -0.0465436726808548, 0.008823499083518982, 0.020171253010630608, 0.012029902078211308, 0.007141257636249065, 0.004886054899543524, -0.026562532410025597, -0.007124160882085562, 0.06844864040613174, -0.0489496663

-------------
## ✏️ Extractors & Datapipeline
- 有些時候，會希望轉換成向量之前，還需要將文句進行進一步的摘要、總結，我們稱之為 Extractors

- 同時，我們會希望建立一個資料處理流程

- 這個處理流程可能包含文檔切割(TokenTextSplitter)、標題擷取(TitleExtractor)、關聯的問題回答擷取(QuestionsAnsweredExtractor)，甚至更多

- 我們可以把這些流程串聯起來(以下叫做transformations)

- 甚至流程可以整合transformations, 存到向量資料庫裡...，建立完整的IngestionPipeline

In [81]:
""" 從sample_pdfs讀取所有相關pdf建立成documents"""
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./datasets/sample_pdfs").load_data()


In [79]:
import os
import nest_asyncio

from llama_index.core.extractors import metadata_extractors
from llama_index.llms.groq import Groq

nest_asyncio.apply()
api_key = os.getenv("GROQ_API_KEY")

# 建立一個LLM model 一樣 by Groq
groq_llm = Groq(model="llama-3.1-70b-versatile", api_key=api_key)

# 使用metadata_extractors上的函數
extractor = metadata_extractors.SummaryExtractor(
                    llm=groq_llm, 
                    summaries=['self'], # 預設是['self']
                    # prompt_template="" # 可不填入，使用預設的prompt，也可自定義
                    )

result = extractor.aextract(documents)

C:\Users\PipiHi\AppData\Local\Temp\ipykernel_45076\2061471635.py:20: RuntimeWarning: coroutine 'Dispatcher.span.<locals>.async_wrapper' was never awaited
  result = extractor.aextract(documents)


In [84]:
import os
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.ingestion import IngestionPipeline

api_key = os.getenv("GROQ_API_KEY")

# 建立一個TokenTextSplitter
text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)


# 建立一個LLM model 一樣 by Groq
groq_llm = Groq(model="llama-3.1-70b-versatile", api_key=api_key)

# 官網範例，也能自定義一個自己的extractor
class CustomExtractor(BaseExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": (
                    node.metadata["document_title"]
                    + "\n"
                    + node.metadata["excerpt_keywords"]
                )
            }
            for node in nodes
        ]
        return metadata_list


# 以下為重點，我們建立一個 transformations:List ，裡面包含了所有的文字處理流程
# 這邊我們使用了TokenTextSplitter, TitleExtractor, QuestionsAnsweredExtractor
transformations = [
    TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128),
    TitleExtractor(nodes=5, llm=groq_llm),
    QuestionsAnsweredExtractor(questions=3, llm=groq_llm),
    # EntityExtractor(prediction_threshold=0.5),
    # SummaryExtractor(summaries=["prev", "self"], llm=llm),
    # KeywordExtractor(keywords=10, llm=llm),
    # CustomExtractor()
]


# 利用IngestionPipeline來執行所有的剛剛定義transformations
pipeline = IngestionPipeline(transformations=transformations)

# 這邊我們使用剛剛定義的documents，並且執行pipeline
nodes = pipeline.run(documents=documents)


100%|██████████| 550/550 [07:24<00:00,  1.24it/s]


In [90]:
# 查看執行完pipeline後其中一筆資訊範例
nodes[1].metadata



{'page_label': '2', 'file_name': 'Deep-Learning-with-PyTorch.pdf'}

In [100]:
import textwrap
from llama_index.core.schema import MetadataMode


#
for node in nodes:
    node.metadata = {
        k: node.metadata[k]
        for k in node.metadata
        if k in ["page_label", "file_name"]
    }


# 取一筆資料範例展示
print(
    "LLM sees:\n",
    (nodes)[12].get_content(metadata_mode=MetadataMode.LLM),
)

LLM sees:
 [Excerpt from document]
page_label: 9
Excerpt:
-----
in the years since the library’s release, it has grown into one of the most prominent deep learning tools for a broad range of applications. PyTorch provides a core data structure, the Tensor, a multidimensional array that has many similarities with NumPy arrays. From that foundation, a laundry list of fea-tures was built to make it easy to get a project up and running, or to design and train investigation into a new neural network architecture. Tensors accelerate mathematical operations (assuming that the appropriate combination of hardware and software is present), and PyTorch has packages for distributed training, worker processes for effi-cient data loading, and an extensive library of common deep learning functions. As Python is for programming, PyTorch is both an excellent introduction to deep learning and a tool usable in professional contexts for real-world, high-level work. W e  b e l i e v e  t h a t  P y T o r c

-------------
## ✏️ IngestionPipeline 
